In [17]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [18]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [19]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1542666, 2)

In [20]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()



,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,MetLife
1,a1W8c000007vDN9EAM,Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,MetLife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,MetLife
3,a1W8c000007vDNBEA2,Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,MetLife
4,a1W8c000007vDNCEA2,Carlos Escarate Y Cia Ltda.,A-0004,0018c00002axQXvAAM,MetLife


In [21]:
Empresas_SF.shape

(7925, 5)

In [22]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [23]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [24]:
pacientes_y_union.shape

(1650117, 4)

In [25]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Vulco S.A.,A-1075,0018c00002cXJZeAAO,MetLife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ,a1W8c0000083laZEAQ,EY - EY Audit SpA,A-4489,0018c00002eTboVAAS,EY
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [26]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,Per,Rut Titular,Dv,Nro Item,nombres,Paterno,Materno,apellidos,Rut Dep,Dv Dep,...,Vig Inicial,Prima Base,Gneg,Desc Gneg,Subgrupo,Desc Subgrupo,Rut Cnt,email,Desc Sucursal Plan,rut
0,25,5476920,2,0,MARCOS,ALMENDRAS,GALLARDO,ALMENDRAS GALLARDO,0,NaN,...,19960801,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,marcos.almendras@laboratoriochile.cl,(TRAMO 5),54769202
1,25,6448016,2,0,MARCELO ANGEL,DEL VALLE,ALE,DEL VALLE ALE,0,NaN,...,19960801,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,Marcelo.DelValle@laboratoriochile.cl,(TRAMO 5),64480162
2,25,6875065,2,0,JUAN EDMUNDO,MARTINEZ,DE LA FUENTE,MARTINEZ DE LA FUENTE,0,NaN,...,20020701,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,Juan.Martinez@laboratoriochile.cl,(TRAMO 5),68750652
3,25,6875065,2,2,ANA,ALARCON,PLAZA,ALARCON PLAZA,5588006,9,...,20020701,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,Juan.Martinez@laboratoriochile.cl,(TRAMO 5),55880069
4,25,7041483,K,0,VICENTE,CARCAMO,ALVAREZ,CARCAMO ALVAREZ,0,NaN,...,20170130,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,(TRAMO 5),7041483K


In [27]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

In [28]:
Pacientes_Archivo.head()

,Per,Rut Titular,Dv,Nro Item,nombres,Paterno,Materno,apellidos,Rut Dep,Dv Dep,...,Prima Base,Gneg,Desc Gneg,Subgrupo,Desc Subgrupo,Rut Cnt,email,Desc Sucursal Plan,rut,RutEstaoNo
0,25,5476920,2,0,Marcos,ALMENDRAS,GALLARDO,Almendras Gallardo,0,NaN,...,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,marcos.almendras@laboratoriochile.cl,(TRAMO 5),54769202,True
1,25,6448016,2,0,Marcelo Angel,DEL VALLE,ALE,Del Valle Ale,0,NaN,...,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,Marcelo.DelValle@laboratoriochile.cl,(TRAMO 5),64480162,True
2,25,6875065,2,0,Juan Edmundo,MARTINEZ,DE LA FUENTE,Martinez De La Fuente,0,NaN,...,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,Juan.Martinez@laboratoriochile.cl,(TRAMO 5),68750652,True
3,25,6875065,2,2,Ana,ALARCON,PLAZA,Alarcon Plaza,5588006,9,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,Juan.Martinez@laboratoriochile.cl,(TRAMO 5),55880069,True
4,25,7041483,K,0,Vicente,CARCAMO,ALVAREZ,Carcamo Alvarez,0,NaN,...,0.0875,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,(TRAMO 5),7041483K,True


In [29]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,Per,Rut Titular,Dv,Nro Item,nombres,Paterno,Materno,apellidos,Rut Dep,Dv Dep,...,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,25,5476920,2,0,Marcos,ALMENDRAS,GALLARDO,Almendras Gallardo,0,NaN,...,True,0018c00002ei7H1AAI,54769202,0018c00002ei7H1AAI,a1W8c0000083lZgEAI,a1W8c0000083lZgEAI,Corp. De Bienestar Trabajadores Laboratorio Chile,A-4434,0018c00002eTcV7AAK,Zurich
1,25,6448016,2,0,Marcelo Angel,DEL VALLE,ALE,Del Valle Ale,0,NaN,...,True,0018c00002ei7H2AAI,64480162,0018c00002ei7H2AAI,a1W8c0000083lZgEAI,a1W8c0000083lZgEAI,Corp. De Bienestar Trabajadores Laboratorio Chile,A-4434,0018c00002eTcV7AAK,Zurich
2,25,6875065,2,0,Juan Edmundo,MARTINEZ,DE LA FUENTE,Martinez De La Fuente,0,NaN,...,True,0018c00002ei7H3AAI,68750652,0018c00002ei7H3AAI,a1W8c0000083lZgEAI,a1W8c0000083lZgEAI,Corp. De Bienestar Trabajadores Laboratorio Chile,A-4434,0018c00002eTcV7AAK,Zurich
3,25,6875065,2,2,Ana,ALARCON,PLAZA,Alarcon Plaza,5588006,9,...,True,0018c00002ei7H4AAI,55880069,0018c00002ei7H4AAI,a1W8c0000083lZgEAI,a1W8c0000083lZgEAI,Corp. De Bienestar Trabajadores Laboratorio Chile,A-4434,0018c00002eTcV7AAK,Zurich
4,25,7041483,K,0,Vicente,CARCAMO,ALVAREZ,Carcamo Alvarez,0,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/reportes/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [31]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel(
     "F:/Empresas SF/reportes/account.xlsx", index=False
)
Subir_pacientes.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21100\798392192.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()


,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingCountry,Identification_Type__c,RecordTypeId
1731,Matias,Said Pacheco,19,jsaid@autopistacentral.cl,35929,Hombre,Chile,RUT,0128c000002Byp4AAC
19069,Mariana T.,Tamayo .,19,tamayolara.carlos@yahoo.com,35651,Mujer,Chile,RUT,0128c000002Byp4AAC
